<a href="https://colab.research.google.com/github/yanamalaanusha/KDM-icp/blob/main/KDM_ICP5_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 75kB/s 
     |████████████████████████████████| 204kB 53.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=106ae575da2fe9b17ef7b2101bb0060f2e0a4ebf1e6760a7a6388c73cdfc4925
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec

In [ ]:
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

In [ ]:
# creating spark dataframe wiht the input data. You can also read the data from file. label represents the 3 documnets (0.0,0.1,0.2)
sentenceData = spark.createDataFrame([
        (0.0, "Welcome to KDM TF_IDF Tutorial."),
        (0.1, "Learn Spark ml tf_idf in today's lab."),
        (0.2, "Spark Mllib has TF-IDF.")
    ], ["label", "sentence"])

In [ ]:
# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

In [ ]:
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Welcome to KDM TF...|[welcome, to, kdm...|
|  0.1|Learn Spark ml tf...|[learn, spark, ml...|
|  0.2|Spark Mllib has T...|[spark, mllib, ha...|
+-----+--------------------+--------------------+



In [ ]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

In [ ]:
# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [ ]:
#displaying the results
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[2,8,13,15,17...|
|  0.1|(20,[2,3,6,7],[0....|
|  0.2|(20,[6,14,15],[0....|
+-----+--------------------+



In [ ]:
spark2 = SparkSession.builder.appName("Ngram Example").getOrCreate()

In [ ]:
#creating dataframe of input
wordDataFrame = spark2.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [ ]:
#creating NGrams with n=2 (two words)
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDataFrame = ngram.transform(wordDataFrame)

In [ ]:
# displaying the results
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [ ]:
# creating spark session
spark3 = SparkSession.builder.appName("Word2Vec Example").getOrCreate()

In [ ]:
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark3.createDataFrame([
    ("McCarthy was asked to analyse the data from the first phase of trials of the vaccine.".split(" "), ),
    ("We have amassed the raw data and are about to begin analysing it.".split(" "), ),
    ("Without more data we cannot make a meaningful comparison of the two systems.".split(" "), ),
    ("Collecting data is a painfully slow process.".split(" "), ),
    ("You need a long series of data to be able to discern such a trend.".split(" "), )
], ["text"])

In [ ]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

In [ ]:
for row in result.collect():
    text, vector = row
    #printing the results
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [McCarthy, was, asked, to, analyse, the, data, from, the, first, phase, of, trials, of, the, vaccine.] => 
Vector: [0.022913106746273115,0.007734941667877138,-0.002375059062615037]

Text: [We, have, amassed, the, raw, data, and, are, about, to, begin, analysing, it.] => 
Vector: [-0.04099500154216702,0.0049125014159541866,-0.0014882660112701931]

Text: [Without, more, data, we, cannot, make, a, meaningful, comparison, of, the, two, systems.] => 
Vector: [0.022649812046438456,0.0013891176248972234,0.01336516855427852]

Text: [Collecting, data, is, a, painfully, slow, process.] => 
Vector: [-0.052166125604084554,0.04153742528121386,-0.018508063097085272]

Text: [You, need, a, long, series, of, data, to, be, able, to, discern, such, a, trend.] => 
Vector: [0.011940793506801129,0.020289071649312974,-0.0013307716076572736]



In [ ]:
# showing the synonyms and cosine similarity of the word in input data
synonyms = model.findSynonyms("data", 5)   # its okay for certain words , real bad for others
synonyms.show(5)

NameError: ignored

In [ ]:
#closing the spark sessions
spark.stop()
spark2.stop()
spark3.stop()





# **Creating 5 separate text files containing text data (blogs,news articles etc)**






In [ ]:
with open("/content/news/t1.txt","r+") as t1:
    doc1 = t1.read()
with open("/content/news/t2.txt","r+") as t2:
    doc2 = t2.read()
with open("/content/news/t3.txt","r+") as t3:
    doc3 = t3.read()
with open("/content/news/t4.txt","r+") as t4:
    doc4 = t4.read()
with open("/content/news/t5.txt","r+") as t5:
    doc5 = t5.read()
# Read all 5 txt files which contains news articles
documents = [doc1,doc2,doc3,doc4,doc5]

# **a.Find out the top10 TF-IDF words for the above input.**




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 
pd.set_option('display.max_columns', 20)

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

          india   england        is       for        on       and        in  \
0      0.016507  0.000000  0.024761  0.033015  0.066029  0.189834  0.115551   
1      0.064143  0.000000  0.053453  0.042762  0.085524  0.181739  0.171048   
2      0.055727  0.324858  0.123837  0.123837  0.111454  0.160989  0.284826   
3      0.156404  0.000000  0.117303  0.061445  0.016758  0.201091  0.290465   
4      0.026009  0.000000  0.013004  0.078027  0.143049  0.117040  0.325111   
Total  0.318791  0.324858  0.332359  0.339086  0.422814  0.850693  1.187002   

             to        of       the  
0      0.330147  0.148566  0.561249  
1      0.310025  0.245882  0.470383  
2      0.241483  0.198140  0.476774  
3      0.206677  0.357496  0.502728  
4      0.143049  0.325111  0.325111  
Total  1.231381  1.275195  2.336246  


# **`b.Find out the top10 TF-IDF words for the lemmatized input`**

In [ ]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

documents = [lemmatized_document1,lemmatized_document2,lemmatized_document3,lemmatized_document4,lemmatized_document5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
          india   england        is       for        on       and        in  \
0      0.016373  0.000000  0.024559  0.032745  0.065491  0.188285  0.114608   
1      0.063690  0.000000  0.053075  0.042460  0.084920  0.180455  0.169840   
2      0.055501  0.323544  0.123336  0.123336  0.111003  0.160337  0.283673   
3      0.155913  0.000000  0.116935  0.061252  0.016705  0.200460  0.289553   
4      0.025838  0.000000  0.012919  0.077514  0.142109  0.116271  0.322975   
Total  0.317315  0.323544  0.330824  0.337307  0.420227  0.845808  1.180650   

             to        of       the  
0      0.327453  0.147354  0.556670  
1      0.307835  0.244145  0.467060  
2      0.240506  0.197338  0.474844  
3      0.206028  0.356373  0.501150  
4      0.142109  0.322975  0.322975  
Total

# **c.Find out the top10TF-IDF words for the n-gram based input.**

In [ ]:
# this function takes document and n int value to generate list of n grams
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

ngram_doc1 = ' '.join([' '.join(x) for x in ngrams(doc1, 3)])
ngram_doc2 = ' '.join([' '.join(x) for x in ngrams(doc2, 3)])
ngram_doc3 = ' '.join([' '.join(x) for x in ngrams(doc3, 3)])
ngram_doc4 = ' '.join([' '.join(x) for x in ngrams(doc4, 3)])
ngram_doc5 = ' '.join([' '.join(x) for x in ngrams(doc5, 3)])

# documents = [ngram_doc1,ngram_doc2,ngram_doc3,ngram_doc4,ngram_doc5]

documents = [doc1,doc2,doc3,doc4,doc5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer( ngram_range=(3,3)) # TfidfVectorizer has inbuilt ngram kwarg which show tfidf for ngrams
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

       to control the  the second test  of troop disengagement  \
0            0.098043          0.00000                0.000000   
1            0.000000          0.00000                0.000000   
2            0.000000          0.10723                0.000000   
3            0.000000          0.00000                0.000000   
4            0.000000          0.00000                0.108825   
Total        0.098043          0.10723                0.108825   

       to monitor the  reports of troop  monitor the situation  \
0            0.000000          0.000000               0.000000   
1            0.000000          0.000000               0.000000   
2            0.000000          0.000000               0.000000   
3            0.000000          0.000000               0.000000   
4            0.108825          0.108825               0.108825   
Total        0.108825          0.108825               0.108825   

       of the farmers  farmers but the  to the virus  in the first  
0    

# **2.Write a simple spark program to read a dataset and find the W2V similar words (words with higher cosine similarity) for the Top10 TF-IDF Words**

In [ ]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1),
        (0.1, doc2),
        (0.2, doc3),
        (0.3, doc4),
        (0.5, doc5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|NEW DELHI: Three ...|[new, delhi:, thr...|
|  0.1|LONDON: Vijay Mal...|[london:, vijay, ...|
|  0.2|The more things c...|[the, more, thing...|
|  0.3|Is the sharp drop...|[is, the, sharp, ...|
|  0.5|Washington: The U...|[washington:, the...|
+-----+--------------------+--------------------+



# **a.Try without NLP**

In [ ]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF without NLP:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[0,1,2,3,4,5...|
|  0.1|(200,[2,3,4,5,6,7...|
|  0.2|(200,[0,1,2,3,4,5...|
|  0.3|(200,[0,1,2,3,4,5...|
|  0.5|(200,[0,2,3,5,6,8...|
+-----+--------------------+

TF-IDF without NLP:
Row(label=0.0, document='NEW DELHI: Three days after the widespread violence in the capital on Republic Day, the farmers protesting against the farm laws at Singhu border clashed with a group of people on Friday.\nThe latter claimed they were locals and accused the farmers of having insulted the national flag. This led to a pitched battle with stone-pelting from both sides and injuries to many.\nThese included the Alipur SHO, Pradeep Paliwal, who was badly wounded when a sword-wielding protester from among the farmers attacked him.\n“Forty-four people, including Ranjeet Singh, a resident of Nawanshahr who assaulted Paliwal with a sword have been arrested. Three other officials who were trying to control the m

# **b.Try with Lemmatization**

In [ ]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

### lemmatizing words from 5 input docs same as previos task

# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, lemmatized_document1),
        (0.1, lemmatized_document2),
        (0.2, lemmatized_document3),
        (0.3, lemmatized_document4),
        (0.5, lemmatized_document5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|NEW DELHI : Three...|[new, delhi, :, t...|
|  0.1|LONDON : Vijay Ma...|[london, :, vijay...|
|  0.2|The more thing ch...|[the, more, thing...|
|  0.3|Is the sharp drop...|[is, the, sharp, ...|
|  0.5|Washington : The ...|[washington, :, t...|
+-----+--------------------+--------------------+



In [ ]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with Lemmatization:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[0,1,2,3,4,5...|
|  0.1|(200,[2,3,4,5,6,7...|
|  0.2|(200,[0,1,2,3,4,5...|
|  0.3|(200,[0,1,2,3,4,5...|
|  0.5|(200,[2,3,4,5,6,7...|
+-----+--------------------+

TF-IDF with Lemmatization:
Row(label=0.0, document="NEW DELHI : Three day after the widespread violence in the capital on Republic Day , the farmer protesting against the farm law at Singhu border clashed with a group of people on Friday . The latter claimed they were local and accused the farmer of having insulted the national flag . This led to a pitched battle with stone-pelting from both side and injury to many . These included the Alipur SHO , Pradeep Paliwal , who wa badly wounded when a sword-wielding protester from among the farmer attacked him . “ Forty-four people , including Ranjeet Singh , a resident of Nawanshahr who assaulted Paliwal with a sword have been arrested . Three other official who were trying to control 

# **c.Try with NGrams**

In [ ]:
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1.split(' ')),
        (0.1, doc2.split(' ')),
        (0.2, doc3.split(' ')),
        (0.3, doc4.split(' ')),
        (0.5, doc5.split(' '))
    ], ["label", "document"])


ngram = NGram(n=2, inputCol="document", outputCol="ngrams")

ngramDataFrame = ngram.transform(documentData)

# applying tf on the words data
hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(ngramDataFrame)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with ngram:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[0,1,2,3,4,5...|
|  0.1|(200,[0,1,2,3,4,5...|
|  0.2|(200,[0,1,2,3,4,5...|
|  0.3|(200,[0,1,2,3,4,5...|
|  0.5|(200,[0,1,2,4,5,6...|
+-----+--------------------+

TF-IDF with ngram:
Row(label=0.0, document=['NEW', 'DELHI:', 'Three', 'days', 'after', 'the', 'widespread', 'violence', 'in', 'the', 'capital', 'on', 'Republic', 'Day,', 'the', 'farmers', 'protesting', 'against', 'the', 'farm', 'laws', 'at', 'Singhu', 'border', 'clashed', 'with', 'a', 'group', 'of', 'people', 'on', 'Friday.\nThe', 'latter', 'claimed', 'they', 'were', 'locals', 'and', 'accused', 'the', 'farmers', 'of', 'having', 'insulted', 'the', 'national', 'flag.', 'This', 'led', 'to', 'a', 'pitched', 'battle', 'with', 'stone-pelting', 'from', 'both', 'sides', 'and', 'injuries', 'to', 'many.\nThese', 'included', 'the', 'Alipur', 'SHO,', 'Pradeep', 'Paliwal,', 'who', 'was', 'badly', 'wounded', 'when', 'a', 'sword-wielding', 'pr